# Computation on value graph

1. Martin Molan
1. Gregor Molan

2021-05-01


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Value-graph" data-toc-modified-id="Value-graph-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Value graph</a></span><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Definition-of-model-topology" data-toc-modified-id="Definition-of-model-topology-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Definition of model topology</a></span></li><li><span><a href="#Definition-of-input-parameters-(graph-edge-weights)" data-toc-modified-id="Definition-of-input-parameters-(graph-edge-weights)-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Definition of input parameters (graph edge weights)</a></span></li><li><span><a href="#Definition-of-input-variables" data-toc-modified-id="Definition-of-input-variables-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Definition of input variables</a></span></li><li><span><a href="#Custom-activation-function" data-toc-modified-id="Custom-activation-function-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Custom activation function</a></span></li><li><span><a href="#Definition-of-learning-rate" data-toc-modified-id="Definition-of-learning-rate-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Definition of learning rate</a></span></li><li><span><a href="#Setup-the-model" data-toc-modified-id="Setup-the-model-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Setup the model</a></span></li><li><span><a href="#Gradients" data-toc-modified-id="Gradients-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Gradients</a></span></li><li><span><a href="#Print-model-information" data-toc-modified-id="Print-model-information-1.9"><span class="toc-item-num">1.9&nbsp;&nbsp;</span>Print model information</a></span></li><li><span><a href="#Final-value-according-to-given-percentage" data-toc-modified-id="Final-value-according-to-given-percentage-1.10"><span class="toc-item-num">1.10&nbsp;&nbsp;</span>Final value according to given percentage</a></span></li><li><span><a href="#Plot-loss-values-according-to-skipped-items-from-Backend-functionality-layer" data-toc-modified-id="Plot-loss-values-according-to-skipped-items-from-Backend-functionality-layer-1.11"><span class="toc-item-num">1.11&nbsp;&nbsp;</span>Plot loss values according to skipped items from Backend functionality layer</a></span><ul class="toc-item"><li><span><a href="#Legend-is-placed-outside-a-plot" data-toc-modified-id="Legend-is-placed-outside-a-plot-1.11.1"><span class="toc-item-num">1.11.1&nbsp;&nbsp;</span>Legend is placed outside a plot</a></span></li></ul></li></ul></li><li><span><a href="#Additional/Optional-information" data-toc-modified-id="Additional/Optional-information-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Additional/Optional information</a></span><ul class="toc-item"><li><span><a href="#Possible-activation-function" data-toc-modified-id="Possible-activation-function-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Possible activation function</a></span></li></ul></li></ul></div>

## Value graph

### Imports

In [ ]:
from keras.layers import Activation, Concatenate, Dense, Input
from keras.models import Model
import numpy as np
import tensorflow as tf
import keras
import keract; 
#   from keract import get_activations;
#   from keract import get_gradients_of_activations;

In [ ]:
tf.compat.v1.disable_eager_execution()
#tf.compat.v1.enable_eager_execution()
#tf.keras.backend.set_floatx('float64')

### Definition of model topology

In [ ]:
class VD(keras.layers.Layer):
    def __init__(self, units=32, input_dim1=32, input_dim2 = 32):
        super(VD, self).__init__()
        
        #input1 -> requirements
        #input2 -> backend
        
        # weights for requirement signal
        # trainable
        w_init1 = tf.ones_initializer()
        self.w1 = tf.Variable(
            initial_value=w_init1(shape=(input_dim1, units), dtype="float32"),
            trainable=True,
        )
        
        # weights for backend signal
        # trainable
        w_init2 = tf.random_normal_initializer()
        self.w2 = tf.Variable(
            initial_value=w_init2(shape=(input_dim2, units), dtype="float32"),
            trainable=True,
        )
        
        # bias
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=(units,), dtype="float32"), trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs[0], self.w1) + self.b - tf.matmul(inputs[0], self.w1)* tf.matmul(inputs[1], self.w2)

### Definition of input parameters (graph edge weights)

In [ ]:
my_weights = [
   np.array([
        [0.1, 0.2, 0.2, 0.5],
        [  0, 0.4, 0,   0.6],
        [0.7, 0.3, 0,   0  ],
        [  0, 0.3, 0.3, 0.4]
   ]),
   np.array([
        [ 0.6,  0.4, 0,   0  ],
        [ 0.6,  0.6, 0.8, 0.8],
        [ 0.1,  0.1, 0.9, 0  ],
        [ 0,    0,   0,   0.8],
        [ 1,    1,   1,   1  ]
   ]),
   np.array([  0.00, 0.00, 0.00, 0.00]),
   np.array([
        [0.05, 0.00, 0.95],
        [0.3,  0.3,  0.4],
        [0.4,  0.0,  0.6],
        [0.3,  0.0,  0.7]]),
   np.array([0., 0., 0.]),
   np.array([[0.8],
             [0.5],
             [  1]]),
   np.array([0])
]

### Definition of input variables

In [ ]:
req_data = np.array([[0.4,0.133,0.2,0.267]]);
back_data = np.array([[0,0,0,0,0]]);
tags_data = np.array([[0]]);

### Custom activation function

In [ ]:
# NOTE: percentage is global variable!
def trade_off(x):
    n = 1 / (1-percentage);
    return(x - x**n/n)

### Definition of learning rate

In [ ]:
# Definition of learning rate for "Gradient descent (with momentum) optimizer"
LR = 0.01;

### Setup the model

In [ ]:
#
# Input:
#  percentage
#  my_weights

# input global variable 
#  percentage ... percentage in activation function

# global variables used for setup custom layer names of the model 
relut_name = 'relu_activ_func';   
tradeoff_name = 'trade_off_activ_func';
output_name = 'output_layer';

# Input parameters
#  my_weights ... graph edge weights, topology of the model
#
def setup_model(my_weights):
    
    # the following variables can have arbitrary values (but distinguish)
    #   for custom layer names of the model
    requ1_name = 'requirement_layer_1';
    requ2_name = 'requirement_layer_2';
    back1_name = 'backend_layer_1';
    back2_name = 'backend_layer_2';
    edge_name  = 'edge_layer';
    
    # --- Model definition ---
    # define two sets of inputs
    input_req1 = Input(shape=(2,),name=requ1_name)
    input_req2 = Input(shape=(2,),name=requ2_name)
    input_req = Concatenate()([input_req1, input_req2])

    # define backend functionalities
    input_backend1 = Input(shape=(2,), name = back1_name)
    input_backend2 = Input(shape=(3,), name = back2_name)
    input_backend = Concatenate()([input_backend1, input_backend2])

    input_req = Input(shape=(4,),name=requ1_name)
    input_backend = Input(shape=(5,), name = back1_name)

    # define middle level
    middle = VD(4, 4, 5)([input_req,input_backend])
    middle = Activation('relu', name=relut_name)(middle)
    middle = Activation(trade_off, name=tradeoff_name)(middle)
    edge = Dense(3, name = edge_name)(middle)

    # define ouput level
    output = Dense(1, name = output_name)(edge)
    mymodel = Model(inputs=[input_req, input_backend], outputs=output)
    opt = keras.optimizers.SGD(learning_rate=LR)

    # --- Model compile ---
    mymodel.compile(optimizer=opt, loss='mse');
    
    # --- Set model weights ---
    mymodel.set_weights(my_weights);
    
    return(mymodel);
        

### Gradients

In [ ]:
# setup the model 'mymodel' with given 'percentage' for activation function
percentage = 0.8;
mymodel = setup_model(my_weights);

In [ ]:
# Defined in previous cells: req_data, back_data, tags_data

percentage = 0.8

# Proposed (pecentage) input value for backend functionality (B1, B2, B3, B4, B5):
prop_percentage = 1; # no backend functionality
prop_percentage = 0; # all backend functionality are presented

prop_percentage_array = [prop_percentage]*5;
dtc = keract.get_gradients_of_activations(
        mymodel, 
        [req_data, np.array([prop_percentage_array])],
        tags_data,
        output_format='simple');
dtc

In [ ]:
# get value for key 'vd_1' or 'vd_2' or 'vd_3' or any 'vd_x' from the model calculation
dtc_vd_value = next(vd_value for my_key,vd_value in dtc.items() if 'vd' in my_key);
dtc_vd_value

In [ ]:
# it is supposed that keract.get_... is called 8 time before
#   ... and vd == vd_8

core_grad = dtc_vd_value*dtc[relut_name]*dtc[tradeoff_name];
core_grad

In [ ]:
print("prop_percentage =",prop_percentage);
my_weights[0].dot(core_grad.T)

In [ ]:
print("prop_percentage =",prop_percentage);
my_weights[1].dot(core_grad.T)

### Print model information

In [ ]:
mymodel.summary();

In [ ]:
from keras.utils.vis_utils import plot_model;
plot_model(mymodel, show_shapes=True, show_layer_names=True, rankdir = 'LR', to_file = 'graph_value.pdf');
plot_model(mymodel, show_shapes=True, show_layer_names=True, rankdir = 'LR') # print to Jupyter notebokk

### Final value according to given percentage

In [ ]:
requirement_name = ['Quality of life','Cardial','Posture','Activity'];
backend_name = ['Smart shoes', 'Smart watch', 'Smart phone', 'O2','Cloud infrastructure'];

def loss_value(mymodel, req_data, i_array):
    return_loss_value = keract.get_activations(mymodel, [req_data, np.array([i_array])]);
    return(return_loss_value);


#
# Input global variable: percentage
#
def print_final_value_perctentage(loss_percentage_array):
    level_0_names = [requirement_name, backend_name]; 
    # we are eliminating only bavckend functionalities
    level_0_names = [backend_name];
    
    # set up the model 'mymodel' 
    mymodel = setup_model(my_weights);
    
    # get output_name value without output_name type
    print("{:.2f}".format(loss_value(mymodel, 
                                     req_data, [0]*5)[output_name][0][0] 
                         ),
          ': Final value with all backend functionalities');

    for level_0_functionality in range(0,len(level_0_names)):
        j = int(10 * percentage );
    #    print('Percentage :',percentage*100,'%')
        
    #    print(level_0_functionality,level_0_names[level_0_functionality])
        loss_percentage_array[0][j] = round(loss_value(mymodel, req_data, [0]*5)[output_name][0][0], 2);
    #    print(' Final value with all backend functionalities:',loss_percentage_array[0][j])

        loss_percentage_array[0][0] = 'Final value';
        
    
        for i in range(0,len(level_0_names[level_0_functionality])):
            funct_array    = [0]*len(level_0_names[level_0_functionality]);
            funct_array[i] = 1;
            
            # get name of backend functionality
            loss_percentage_array[i+1][0] = level_0_names[level_0_functionality][i];
            
            # get output_name value without output_name type
            loss_percentage_array[i+1][j] = round(loss_value(mymodel, req_data, funct_array)[output_name][0][0], 2);
            print('  ',
    #             i,
                  # get output_name value without output_name type
                  "{:.2f}".format( loss_value(mymodel, req_data, funct_array)[output_name][0][0] ),
                  ':',
                  'Without',
                  loss_percentage_array[i+1][0],
    #              funct_array,
                );

In [ ]:
# initialize 
loss_percentage_array = [[0 for i in range(10)] for j in range(6)];

# Note: percentage is global!
for j in range(0,9):
    percentage = (j+1)/10
    # Write values
    print("\n{:n}% : defined percentage for trade off function".format( percentage*100 ));
    print("-"*45);
    
    # percentage is parameter defined as GlOBAL variable
    print_final_value_perctentage(loss_percentage_array); 

In [ ]:
loss_percentage_array

### Plot loss values according to skipped items from Backend functionality layer

In [ ]:
import matplotlib.pyplot as plt

cm = 1/2.54;
back_names = [loss_percentage_array[5][0]];
plt.plot([*range(10,100,10)], loss_percentage_array[5][1:10]);
for i in range(0,5):
    print(loss_percentage_array[i][0]);
    back_names.append(loss_percentage_array[i][0]);
    plt.plot([*range(10,100,10)], loss_percentage_array[i][1:10]);
plt.legend(back_names);

plt.savefig('value_retained-1.pdf');

#### Legend is placed outside a plot

In [ ]:
import matplotlib.pyplot as plt

# define plot size to place legend in the figure during 'savefig'
fig = plt.figure(figsize=(7,4));

cm = 1/2.54;
back_names = [loss_percentage_array[5][0]];
plt.plot([*range(10,100,10)], loss_percentage_array[5][1:10]);
for i in range(0,5):
    print(loss_percentage_array[i][0]);
    back_names.append(loss_percentage_array[i][0]);
    plt.plot([*range(10,100,10)], loss_percentage_array[i][1:10]);

# put legend outside a plot
legend_x = -0.5;
legend_y = 0.5;
plt.legend(back_names, loc='center left', bbox_to_anchor=(legend_x, legend_y))

# bbox_inches = 'tight' : try to figure out the tight bbox of the figure
plt.savefig('value_retained-left_legend.pdf', bbox_inches = 'tight');

In [ ]:
import matplotlib.pyplot as plt

# define plot size to place legend in the figure during 'savefig'
fig = plt.figure(figsize=(7,4));

cm = 1/2.54;
back_names = [loss_percentage_array[5][0]];
plt.plot([*range(10,100,10)], loss_percentage_array[5][1:10]);
for i in range(0,5):
    print(loss_percentage_array[i][0]);
    back_names.append(loss_percentage_array[i][0]);
    plt.plot([*range(10,100,10)], loss_percentage_array[i][1:10]);

# put legend outside a plot
legend_x = 1;
legend_y = 0.5;
plt.legend(back_names, loc='center left', bbox_to_anchor=(legend_x, legend_y))

# bbox_inches = 'tight' : try to figure out the tight bbox of the figure
plt.savefig('value_retained.pdf', bbox_inches = 'tight');

## Additional/Optional information

### Possible activation function
Requirements for activation function $y()$
1. $y(0) = 0  $
1. $y'(0) = 1 $
1. $y(x) \le x, \; x \in [0,1] $
1. $y(1) = percentage $

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

fig = plt.figure(figsize=(14,7))
gs = gridspec.GridSpec(nrows=1, ncols=2)

#
# The first trade_off function
#
def trade_off_sample(x,percentage):
    p = (np.tanh(1) - percentage)/(np.tanh(1)-1)
    return(p*(x - np.tanh(x))+np.tanh(x))

# 100 linearly spaced numbers
x = np.linspace(0,1,100)

# setting the axes at the centre

ax0 = fig.add_subplot(gs[0, 0])
ax0.spines['left'].set_position('zero')
ax0.spines['bottom'].set_position('zero')
ax0.spines['right'].set_color('none')
ax0.spines['top'].set_color('none')
ax0.xaxis.set_ticks_position('bottom')
ax0.yaxis.set_ticks_position('left')

# plot the indentiy function red
y0 = x
plt.plot(x,y0, 'r')
# plot the 10% trade_off function function cyan
y1 = trade_off_sample(x,0.1)
plt.plot(x,y1, 'c')
# plot all trade_off functions with percentage 20-90 blue
for i in range(2,10):
    plt.plot(x,trade_off_sample(x,i/10), 'b')

plt.axvline(x=1, color='k', linestyle=(0, (1, 7)))
plt.axhline(y=1, color='k', linestyle=(0, (1, 7)))

#
# The second trade_off function
#
n=2;
def trade_off_sample(x,percentage):
    n = 1 / (1-percentage);
    y = x - x**n/n;
    return(y)

# 100 linearly spaced numbers
x = np.linspace(0,1,100)

# setting the axes at the centre

ax1 = fig.add_subplot(gs[0, 1])
ax1.spines['left'].set_position('zero')
ax1.spines['bottom'].set_position('zero')
ax1.spines['right'].set_color('none')
ax1.spines['top'].set_color('none')
ax1.xaxis.set_ticks_position('bottom')
ax1.yaxis.set_ticks_position('left')

# plot the indentiy function red
y0 = x
plt.plot(x,y0, 'r')
# plot the 10% trade_off function function cyan
y1 = trade_off_sample(x,0.1)
plt.plot(x,y1, 'c')
# plot all trade_off functions with percentage 20-90 blue
for i in range(2,10):
    plt.plot(x,trade_off_sample(x,i/10), 'b')

plt.axvline(x=1, color='k', linestyle=(0, (1, 7)))
plt.axhline(y=1, color='k', linestyle=(0, (1, 7)))

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

fig = plt.figure(figsize=(14,7))
gs = gridspec.GridSpec(nrows=1, ncols=2)


#
# The second trade_off function
#
n=2;
def trade_off_sample(x,percentage):
    n = 1 / (1-percentage);
    y = x - x**n/n;
    return(y)

# 100 linearly spaced numbers
x = np.linspace(0,1,100)

# setting the axes at the centre

ax1 = fig.add_subplot(gs[0, 1])
ax1.spines['left'].set_position('zero')
ax1.spines['bottom'].set_position('zero')
ax1.spines['right'].set_color('none')
ax1.spines['top'].set_color('none')
#ax1.xaxis.set_ticks_position('bottom')
#ax1.yaxis.set_ticks_position('left')

# plot the indentiy function red
y0 = x
plt.plot(x,y0, 'r')
# plot the 80% trade_off function function blue
y1 = trade_off_sample(x,0.8)
plt.plot(x,y1, 'b')

plt.xticks(np.arange(0.2, 1.1, step=0.2))  # Set label locations.
plt.yticks(np.arange(0.2, 1.1, step=0.2))  # Set label locations.

plt.axvline(x=1, ymin=0.05, color='k', linestyle=(0, (1, 7)))
plt.axhline(y=1, xmin=0.05, color='k', linestyle=(0, (1, 7)))
plt.axhline(y=0.8, xmin=0.05, color='k', linestyle=(0, (1, 7)))

#plt.tight_layout()
plt.show()